In [2]:
import pandas as pd
import numpy as np
import csv
import json
import string
import random
import re
from scipy.stats import pearsonr
from matplotlib import pyplot

# np.random.seed(1234)
# random.seed(1234)

In [3]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
import string

porter = PorterStemmer()
punctuation_dictionary = {s:None for s in list(string.punctuation)}
punctuation_dictionary["-"] = "_"

punctuation_translator = str.maketrans(punctuation_dictionary)

def text_cleaner(text, punctuation_translator, stemmer):
    text = "".join(re.split('xa0',text))
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    text = porter.stem(text)
    return(text)

In [3]:
#SubSpeech_predicted = pd.read_csv('SubSpeech_predicted_update.csv')

In [286]:
#SubSpeech_predicted = pd.read_csv('SubSpeech_predicted_update_new.csv')
SubSpeech_predicted = pd.read_csv('SubSpeech_predicted_final.csv')

In [287]:
len(SubSpeech_predicted.Subspeech_index)

16729

In [288]:
SubSpeech_predicted.Pop_class.value_counts()

0    16302
1      427
Name: Pop_class, dtype: int64

In [289]:
Speech = pd.read_csv('CampaignSpeechMerged_cleaned.csv')

In [290]:
len(Speech.Speech_index), len(SubSpeech_predicted.Speech_index.unique())

(4364, 4364)

In [291]:
Polls = pd.read_csv('Polls.csv')

In [292]:
AllCandidate = pd.read_csv('AllCandidates.csv')

In [293]:
#Speech.head()

In [294]:
AllCandidate.head()

,Candidate,Year,Party,Third_can,Final,P2P
0,Donald J. Trump,2016,R,0,1,0
1,Ted Cruz,2016,R,0,0,0
2,Marco Rubio,2016,R,0,0,0
3,John Kasich,2016,R,0,0,0
4,Ben Carson,2016,R,0,0,0


In [295]:
#corr, _ = pearsonr(SubSpeech_predicted['Pop_prob0'], SubSpeech_predicted['Pop_prob'])
#print('Pearsons correlation: %.3f' % corr)

In [296]:
#SubSpeech_predicted[SubSpeech_predicted.Pop.isna()].sort_values('Dif').head(20)

In [297]:
#SubSpeech_predicted['CandiYear'].unique()

In [298]:
Speech.Month.unique()

array(['November', 'October', 'September', 'August', 'July', 'June',
       'March', 'May', 'April', 'February', 'January', 'December', '10.0',
       '11.0', nan, '7.0', '9.0', '8.0'], dtype=object)

## Speech Level

In [299]:
SubSpeech_predicted.Month.replace({'7.0': 'July', '8.0': 'August', '9.0': 'September', '10.0': 'October', '11.0': 'November', },
                        inplace=True)

In [300]:
Speech.Month.replace({'7.0': 'July', '8.0': 'August', '9.0': 'September', '10.0': 'October', '11.0': 'November', },
                        inplace=True)

In [301]:
SubSpeech_predicted['CandiYear'] = SubSpeech_predicted.Candidate + '_' + SubSpeech_predicted.Year.astype(str) 

In [302]:
CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [303]:
CandiPop_SubSpeech = SubSpeech_predicted[SubSpeech_predicted.CandiYear.isin(CanditateYear)]

In [304]:
len(CandiPop_SubSpeech.Speech_index.unique())

3495

In [305]:
CandiPop_SubSpeech = CandiPop_SubSpeech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                                         'Subspeech_index', 'Title','Type','Pop_class']]                         

In [306]:
CandiPop_SubSpeech.to_csv('Subspeech_Level_Pop_MajorCand.csv', index=False)

In [307]:
#CandiPop_SubSpeech.to_csv('Subspeech_Level_Pop_MajorCand_new.csv', index=False)

In [308]:
Pop_prop = []
SubSpeechCounts = []
for x in Speech.Speech_index:
    Pop_prop.append(np.mean(SubSpeech_predicted.Pop_class[SubSpeech_predicted.Speech_index == x]))
    SubSpeechCounts.append(len(SubSpeech_predicted.Subspeech_index[SubSpeech_predicted.Speech_index == x]))

In [309]:
Pop_Speech = Speech.assign(Pop_prop = Pop_prop)
Pop_Speech = Pop_Speech.assign(SubSpeechCounts = SubSpeechCounts)

In [310]:
#Pop_Speech.head()

### Add speech word counts

In [311]:
Speech['cleantext'] = Speech["Content"].apply(lambda x: text_cleaner(x, punctuation_translator, porter))

In [312]:
Speech['Speech_len'] = [len(x.split(' ')) for x in Speech['cleantext']]

In [313]:
Speech_len = Speech[['Speech_index','Speech_len']]

In [314]:
Pop_Speech = Pop_Speech.merge(Speech_len)

In [315]:
Pop_Speech = Pop_Speech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                         'Title','Type','Pop_prop','SubSpeechCounts','Speech_len']]

In [316]:
Pop_Speech.Month.unique()

array(['November', 'October', 'September', 'August', 'July', 'June',
       'March', 'May', 'April', 'February', 'January', 'December', nan],
      dtype=object)

In [317]:
len(Pop_Speech[Pop_Speech.Year>=1952].Title)

4325

In [318]:
#len(CandiPop_Speech.loc[CandiPop_Speech.Speech_len.isin(list(range(400,7000)))].Speech_len)/len(CandiPop_Speech.Title)

In [319]:
Pop_Speech.to_csv("Speech_Level_Pop_All.csv", index=False)

In [320]:
#Pop_Speech.to_csv("Speech_Level_Pop_All_new.csv", index=False)

### Select Relevant Speeches

In [321]:
set(Polls.Candidate).difference(set(Pop_Speech.Candidate))

{'Alfred M. Landon',
 'Gary Johnson',
 'George Wallace',
 'John Anderson',
 'Ralph Nader',
 'Strom Thurmond'}

In [322]:
Pop_Speech['CandiYear'] = Pop_Speech.Candidate + '_' + Pop_Speech.Year.astype(str) 

In [323]:
CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [324]:
CandiPop_Speech = Pop_Speech[Pop_Speech.CandiYear.isin(CanditateYear)]

In [325]:
len(CandiPop_Speech.Title)

3495

In [326]:
CandiPop_Speech.Speech_len.describe()

count     3495.000000
mean      2167.230615
std       1369.323702
min         25.000000
25%       1140.500000
50%       1984.000000
75%       2905.500000
max      11057.000000
Name: Speech_len, dtype: float64

In [327]:
len(CandiPop_Speech.loc[CandiPop_Speech.Speech_len.isin(list(range(400,7000)))].Speech_len)/len(CandiPop_Speech.Title)

0.9476394849785408

In [328]:
CandiPop_Speech.sort_values(by=['Speech_len'], ascending=True).tail(5)

,Candidate,Chanel,City,Year,Month,Day,Speech_index,Title,Type,Pop_prop,SubSpeechCounts,Speech_len,CandiYear
117,Donald J. Trump,NaN,NaN,2016,August,5.0,Speech117,\nRemarks at the KI Convention Center in Green...,Campaign Documents,0.047619,21,9261,Donald J. Trump_2016
2224,Hubert H. Humphrey,STUMP,BUFFALO,1968,September,17.0,Speech3099,NaN,SPEECH,0.000000,8,9280,Hubert H. Humphrey_1968
191,Hillary Clinton,NaN,NaN,2016,February,3.0,Speech191,\nRemarks at the CNN Democratic Presidential T...,Campaign Documents,0.000000,14,9327,Hillary Clinton_2016
119,Donald J. Trump,NaN,NaN,2016,July,27.0,Speech119,"\nNews Conference in Doral, Florida\n",Campaign Documents,0.000000,32,9727,Donald J. Trump_2016
2248,Hubert H. Humphrey,STUMP,PORTLAND,1968,September,27.0,Speech3123,NaN,SPEECH,0.000000,12,11057,Hubert H. Humphrey_1968


In [329]:
CandiPop_Speech.to_csv("Speech_Level_Pop_MajorCand.csv", index = False)

In [330]:
#CandiPop_Speech.to_csv("Speech_Level_Pop_MajorCand_new.csv", index = False)

## Candidate level

In [331]:
CandiPop = SubSpeech_predicted.groupby(['Candidate','Year']).Pop_class.mean().reset_index()

In [332]:
CandiCounts = SubSpeech_predicted.groupby(['Candidate','Year']).Subspeech_index.count().reset_index()

In [333]:
CandiSpeechCounts = Speech.groupby(['Candidate','Year']).Speech_index.count().reset_index()

In [334]:
CandiPop = CandiPop.merge(CandiCounts).merge(CandiSpeechCounts)
CandiPop.head()

,Candidate,Year,Pop_class,Subspeech_index,Speech_index
0,Adlai Stevenson,1952,0.000000,409,186
1,Adlai Stevenson,1956,0.004032,248,75
2,Al Smith,1928,0.000000,9,1
3,"Albert Gore, Jr.",1999,0.000000,7,1
4,"Albert Gore, Jr.",2000,0.011628,86,14


In [335]:
len(CandiPop.Subspeech_index)

164

In [336]:
CandiPop.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
                      inplace = True)

In [337]:
#CandiPop.head()

In [338]:
CandiPop.to_csv('AllCandidateLevel.csv', index=False)

In [339]:
#CandiPop = pd.read_csv('AllCandidateLevel.csv')

In [340]:
CandiPop['CandiYear'] = CandiPop.Candidate + '_' + CandiPop.Year.astype(str) 

In [341]:
AllCandidate['CandiYear'] = AllCandidate.Candidate + '_' + AllCandidate.Year.astype(str) 

In [342]:
set(AllCandidate.Candidate).difference(set(CandiPop.Candidate))

{'Alfred M. Landon',
 'Gary Johnson',
 'George Wallace',
 'Jim Gilmore',
 'John Anderson',
 'Ralph Nader',
 'Strom Thurmond'}

In [343]:
#set(CandiPop.CandiYear).difference(set(AllCandidate.CandiYear))

In [344]:
AllCandidate.head()

,Candidate,Year,Party,Third_can,Final,P2P,CandiYear
0,Donald J. Trump,2016,R,0,1,0,Donald J. Trump_2016
1,Ted Cruz,2016,R,0,0,0,Ted Cruz_2016
2,Marco Rubio,2016,R,0,0,0,Marco Rubio_2016
3,John Kasich,2016,R,0,0,0,John Kasich_2016
4,Ben Carson,2016,R,0,0,0,Ben Carson_2016


In [345]:
CandiPop_All = CandiPop.merge(AllCandidate[['Party','Final','P2P','CandiYear']])

In [346]:
#CandiPop.sort_values('Pop_prop').tail(30)
CandiPop_All = CandiPop_All[CandiPop_All.Speech_counts > 4]

In [347]:
CandiPop_All.to_csv('AllCandidate_Year_pop.csv', index = False)

In [348]:
AllCandi_SubSpeech = SubSpeech_predicted[SubSpeech_predicted.CandiYear.isin(CandiPop_All.CandiYear)]
AllCandi_SubSpeech = AllCandi_SubSpeech[['Candidate','Chanel','City','Year','Month','Day','Speech_index',
                                         'Subspeech_index', 'Title','Type','Pop_class', 'CandiYear']]

In [349]:
AllCandi_SubSpeech = AllCandi_SubSpeech.merge(AllCandidate[['Party','Final','P2P','CandiYear']])

In [350]:
AllCandi_SubSpeech.Party.describe()

count     12948
unique        2
top           D
freq       7442
Name: Party, dtype: object

In [351]:
AllCandi_SubSpeech['YearCandi'] = AllCandi_SubSpeech.Year.astype(str) + ' ' + AllCandi_SubSpeech.Candidate 

In [352]:
AllCandi_SubSpeech.to_csv('AllCandidate_SubSpeech_Pop.csv', index = False)

## Select final candidates

In [353]:
# CandiPop = pd.read_csv('AllCandidateLevel.csv')

In [354]:
#CandiPop.head()

In [355]:
#CanditateYear = Polls.Candidate + '_' + Polls.Year.astype(str)

In [356]:
#CandiPop[CandiPop.Speech_index >5].sort_values('Pop_class').tail(30)

In [357]:
#CandiPop['CandiYear'] = CandiPop.Candidate + '_' + CandiPop.Year.astype(str) 

In [358]:
CandiPop_final = CandiPop[CandiPop.CandiYear.isin(CanditateYear)]

In [359]:
CandiPop_final.head()

,Candidate,Year,Pop_prop,Subspeech_counts,Speech_counts,CandiYear
0,Adlai Stevenson,1952,0.000000,409,186,Adlai Stevenson_1952
1,Adlai Stevenson,1956,0.004032,248,75,Adlai Stevenson_1956
4,"Albert Gore, Jr.",2000,0.011628,86,14,"Albert Gore, Jr._2000"
8,Barack Obama,2008,0.083942,822,185,Barack Obama_2008
12,Barack Obama,2012,0.011650,1030,205,Barack Obama_2012


In [360]:
#CandiPop_final.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
#                      inplace = True)

In [361]:
#CandiPop_final.sort_values('Pop_prop', ascending= False).head(25)

In [362]:
CandiPop_final[CandiPop_final.Year.isin(list(range(1952,1997)))].sort_values('Pop_prop', ascending= False)

,Candidate,Year,Pop_prop,Subspeech_counts,Speech_counts,CandiYear
48,George McGovern,1972,0.050781,256,98,George McGovern_1972
68,Jimmy Carter,1976,0.019737,152,55,Jimmy Carter_1976
117,Richard Nixon,1968,0.014045,356,67,Richard Nixon_1968
36,Dwight Eisenhower,1952,0.009780,409,230,Dwight Eisenhower_1952
99,Michael S. Dukakis,1988,0.007692,260,66,Michael S. Dukakis_1988
95,Lyndon B. Johnson,1964,0.006608,454,111,Lyndon B. Johnson_1964
44,George Bush,1988,0.005181,193,43,George Bush_1988
150,Walter F. Mondale,1984,0.004149,241,67,Walter F. Mondale_1984
1,Adlai Stevenson,1956,0.004032,248,75,Adlai Stevenson_1956
156,William J. Clinton,1992,0.003195,313,76,William J. Clinton_1992


In [363]:
len(CandiPop_final.Candidate)

41

In [364]:
CandiPop_final.groupby(['Year']).Pop_prop.mean()

Year
1936    0.000000
1940    0.000000
1944    0.000000
1948    0.000000
1952    0.004890
1956    0.002016
1960    0.000971
1964    0.003304
1968    0.007695
1972    0.025391
1976    0.009868
1980    0.000000
1984    0.003437
1988    0.006437
1992    0.001065
1996    0.000882
2000    0.025814
2004    0.001458
2008    0.047933
2012    0.008083
2016    0.139651
Name: Pop_prop, dtype: float64

In [365]:
CandiPop_final.to_csv('CandidateMerged_Level.csv', index= False)

In [366]:
#CandiPop_final.to_csv('CandidateMerged_Level_new.csv', index= False)

In [367]:
#CandiPop_final = pd.read_csv('CandidateMerged_Level_new.csv') 

## Candidate Month

In [368]:
SubSpeech_predicted.Month.unique()

array(['November', 'October', 'September', 'August', 'July', 'June',
       'March', 'May', 'April', 'February', 'January', 'December', nan],
      dtype=object)

In [369]:
CandiPopMonth = SubSpeech_predicted.groupby(['Candidate','Year','Month']).Pop_class.mean().reset_index()

In [370]:
CandiCountsMonth = SubSpeech_predicted.groupby(['Candidate','Year', 'Month']).Subspeech_index.count().reset_index()

In [371]:
CandiSpeechCountsMonth = Speech.groupby(['Candidate','Year','Month']).Speech_index.count().reset_index()

In [372]:
CandiPopMonth = CandiPopMonth.merge(CandiCountsMonth).merge(CandiSpeechCountsMonth)
CandiPopMonth.head()

,Candidate,Year,Month,Pop_class,Subspeech_index,Speech_index
0,Adlai Stevenson,1952,August,0.0,10,4
1,Adlai Stevenson,1952,July,0.0,3,2
2,Adlai Stevenson,1952,November,0.0,10,4
3,Adlai Stevenson,1952,October,0.0,271,124
4,Adlai Stevenson,1952,September,0.0,109,46


In [373]:
CandiPopMonth.rename(columns={'Pop_class': 'Pop_prop', 'Subspeech_index': 'Subspeech_counts', 'Speech_index': 'Speech_counts' }, 
                      inplace = True)

In [374]:
CandiPopMonth['CandiYear'] = CandiPopMonth.Candidate + '_' + CandiPopMonth.Year.astype(str) 

In [375]:
CandiPopMonth = CandiPopMonth[CandiPopMonth.CandiYear.isin(CanditateYear)]

In [376]:
CandiPopMonth.to_csv('CandidateYearMonth_Level_Pop.csv', index=False)

In [377]:
#CandiPopMonth.to_csv('CandidateYearMonth_Level_Pop_new.csv', index=False)

## Average Year Level 

In [378]:
PopYear_level = SubSpeech_predicted.groupby(['Year']).Pop_class.mean().reset_index()

In [379]:
PopYear_level.Pop_class.describe()

count    48.000000
mean      0.013444
std       0.035219
min       0.000000
25%       0.000000
50%       0.000000
75%       0.007005
max       0.181818
Name: Pop_class, dtype: float64

In [98]:
PopYear_level.to_csv('PopYear_level.csv', index =False)

In [99]:
len(SubSpeech_predicted[SubSpeech_predicted.Candidate == "Donald J. Trump"].Speech_index.unique())

87